In [ ]:
from pathlib import Path

from loguru import logger
from huggingface_hub import snapshot_download

DATASET_DIR = Path.cwd()
CORPUS_NAMES = ["statpearls", "textbooks"]
DATASET_DIR

In [ ]:
def download_corpus(corpus_name: str, use_subset: bool = True):
    """
    Download the specified corpus from
    https://huggingface.co/datasets/khoaguin/medical-corpus
    """
    if use_subset:
        DATASET_PATH = DATASET_DIR / "subsets" / corpus_name
        allow_patterns = f"subsets/{corpus_name}/*"
    else:
        DATASET_PATH = DATASET_DIR / "full" / corpus_name
        allow_patterns = f"{corpus_name}/*"

    if DATASET_PATH.exists():
        logger.warning(
            f"Corpus {corpus_name} already exists at {DATASET_PATH}. Skipping download."
        )
        _log_dataset_structure(DATASET_PATH, corpus_name)
        return

    logger.info(f"Downloading {corpus_name} to {DATASET_PATH}...")
    snapshot_download(
        repo_id="khoaguin/medical-corpus",
        repo_type="dataset",
        local_dir=DATASET_DIR,
        allow_patterns=allow_patterns,
    )
    _log_dataset_structure(DATASET_PATH, corpus_name)


def _log_dataset_structure(dataset_path: Path, corpus_name: str):
    """Log dataset structure in a clean tree format with sizes"""

    logger.info(f"📊 Dataset structure for {corpus_name}:")

    def get_dir_stats(path):
        """Get file count and size for a directory"""
        files = list(path.rglob("*"))
        file_count = sum(1 for f in files if f.is_file())
        total_size_mb = sum(f.stat().st_size for f in files if f.is_file()) / (
            1024 * 1024
        )
        return file_count, total_size_mb

    def format_size(size_mb):
        """Format size in MB or KB"""
        if size_mb < 1:
            return f"{size_mb * 1024:.2f} KB"
        return f"{size_mb:.2f} MB"

    # Process mock and private directories
    for subdir_name in ["mock", "private"]:
        subdir = dataset_path / subdir_name
        if subdir.exists():
            file_count, size_mb = get_dir_stats(subdir)
            logger.info(
                f"  📁 {subdir_name}/ - {file_count} files ({format_size(size_mb)})"
            )

            # Show chunk subdirectory with size
            chunk_subdir = subdir / "chunk"
            if chunk_subdir.exists():
                chunk_files = list(chunk_subdir.glob("*.jsonl"))
                if chunk_files:
                    chunk_size = sum(f.stat().st_size for f in chunk_files) / (
                        1024 * 1024
                    )
                    logger.info(
                        f"    📁 chunk/ - {len(chunk_files)} jsonl files  ({format_size(chunk_size)})"
                    )

            # Show key files with sizes
            for filename in ["all_doc_ids.npy", "faiss.index", "README.md"]:
                file_path = subdir / filename
                if file_path.exists():
                    file_size_mb = file_path.stat().st_size / (1024 * 1024)
                    logger.info(f"    📄 {filename} ({format_size(file_size_mb)})")

    # Total summary
    total_files = sum(1 for _ in dataset_path.rglob("*") if _.is_file())
    total_size_mb = sum(
        _.stat().st_size for _ in dataset_path.rglob("*") if _.is_file()
    ) / (1024 * 1024)
    logger.info(f"✅ Total: {total_files} files, {format_size(total_size_mb)}")

In [ ]:
for corpus in CORPUS_NAMES:
    download_corpus(corpus, use_subset=True)